In [1]:
import json
import os
import numpy as np
from pymilvus import connections, MilvusClient, FieldSchema, CollectionSchema, DataType, Collection, AnnSearchRequest, RRFRanker
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

from pathlib import Path
# 如果在 Notebook 中运行，__file__ 不存在，改用当前工作目录
try:
    SCRIPT_DIR = Path(__file__).resolve().parent
except NameError:
    SCRIPT_DIR = Path.cwd()  # 使用当前工作目录
print(SCRIPT_DIR)

# 1. 初始化设置
COLLECTION_NAME = "my_test_dragon_hybrid_demo"
MILVUS_URI = "http://localhost:19530"  # 服务器模式
DATA_PATH = SCRIPT_DIR / "../../data/C4/metadata/dragon.json"  # 相对路径
BATCH_SIZE = 50

/root/miniconda/envs/myrag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/workspace/all-in-rag/code/C4


In [2]:
# 2. 连接 Milvus 并初始化嵌入模型
print(f"--> 正在连接到 Milvus: {MILVUS_URI}")
connections.connect(uri=MILVUS_URI)

--> 正在连接到 Milvus: http://localhost:19530


In [3]:
print("--> 正在初始化 BGE-M3 嵌入模型。。。")
ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
bge_m3_dense_dim = ef.dim["dense"]
print(f"--> 嵌入模型初始化完成。密集向量维度: {bge_m3_dense_dim}")


--> 正在初始化 BGE-M3 嵌入模型。。。


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 246723.76it/s]


--> 嵌入模型初始化完成。密集向量维度: 1024



## 建表

{'img_id': 'dragon02', 'path': '../../data/C3/dragon/dragon02.png', 'title': '悬崖上的白龙', 'description': '一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。', 'category': 'western_dragon', 'location': '悬崖海岸', 'environment': '天空'}

In [4]:
mclient = MilvusClient(uri=MILVUS_URI)

# Note: 如果 auto_id=False, 需要自己准备 pk项
# DataNotMatchException: <DataNotMatchException: (code=1, 
# message=The data doesn't match with schema fields, expect 10 list, got 9)>

fields = [
    FieldSchema(name="primary_key", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=100),
    FieldSchema(name="img_id", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="path", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="description", dtype=DataType.VARCHAR, max_length=4096),
    FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="location", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="environment", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=bge_m3_dense_dim)
]

In [10]:


if not mclient.has_collection(COLLECTION_NAME):
    print(f"Collection '{COLLECTION_NAME}' 不存在，请先创建")
    schema = CollectionSchema(fields=fields, description="龙混合检索示例")
    collection = Collection(name=COLLECTION_NAME, schema=schema, consistency_level="Strong")

    sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
    dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
    collection.create_index("sparse_vector", sparse_index)
    collection.create_index("dense_vector", dense_index)
else:
    print(f"Collection: 已经存在 -- 删除 '{COLLECTION_NAME}'")
    mclient.drop_collection(collection_name=COLLECTION_NAME)


Collection 'my_test_dragon_hybrid_demo' 不存在，请先创建


## 加载到内存

🔍 为什么需要加载？

| 操作                           | 是否需要加载   | 说明                            |
| ------------------------------ | -------------- | ------------------------------- |
| `insert()` / `upsert()`        | ❌ 不需要       | 数据直接写入存储层              |
| `create_index()`               | ❌ 不需要       | 索引构建在存储层完成            |
| `search()` / `hybrid_search()` | ✅ **必须加载** | 向量搜索需将数据/索引加载到内存 |
| `query()`（带向量字段）        | ✅ **必须加载** | 涉及向量计算时需要加载          |

> 💡 **Milvus 设计逻辑**：为节省内存资源，集合默认**不自动加载**。需用户显式控制加载/释放（`load()` / `release()`）。

In [21]:
collection = Collection(COLLECTION_NAME)

from pymilvus import utility

print(utility.load_state(collection.name))

print(f"⚠️ 集合 {collection.name} 未加载，正在加载...")
collection.load()
print("✅ 集合加载完成")

NotLoad
⚠️ 集合 my_test_dragon_hybrid_demo 未加载，正在加载...
✅ 集合加载完成


## 插入数据



In [ ]:


if not collection.is_empty:
    print(f"存在数据, 共有 {collection.num_entities} 条")

if collection.is_empty:
    print("不存在数据")
    if not os.path.exists(DATA_PATH):
        raise FileNotFoundError(f"数据文件未找到: {DATA_PATH}")
    with open(DATA_PATH, "r") as f:
        datasets = json.load(f)
    print(datasets[0])

    docs, meta_data = [], []
    for item in datasets:
        parts = [
            item.get("title", ''),
            item.get("description", ''),
            item.get("location", ''),
            item.get("environment", ''),
        ]

        docs.append(' '.join(filter(None, parts)))
        meta_data.append(item)
    print(f"--> 数据加载完成，共 {len(docs)} 条。")

    print("开始生成嵌入向量")
    print(docs)
    embedings = ef(docs)
    sparse_vectors = embedings['sparse']
    dense_vectors = embedings['dense']

    print("--> 正在分批插入数据...")
    data = [
        [str(i) for i in range(len(meta_data))],
        [doc["img_id"] for doc in meta_data],
        [doc["path"] for doc in meta_data],
        [doc["title"] for doc in meta_data],
        [doc["description"] for doc in meta_data],
        [doc["category"] for doc in meta_data],
        [doc["location"] for doc in meta_data],
        [doc["environment"] for doc in meta_data],
        sparse_vectors,
        dense_vectors
    ]

    collection.insert(data)
    collection.flush()
    print(f"已经存在的数据数量: {collection.num_entities}")

不存在数据
{'img_id': 'dragon02', 'path': '../../data/C3/dragon/dragon02.png', 'title': '悬崖上的白龙', 'description': '一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。', 'category': 'western_dragon', 'location': '悬崖海岸', 'environment': '天空'}
--> 数据加载完成，共 6 条。
开始生成嵌入向量
['悬崖上的白龙 一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。 悬崖海岸 天空', '霸王龙的怒吼 史前时代的霸王龙张开血盆大口，发出震天的怒吼。在它身后，几只翼龙在阴沉的天空中盘旋，展现了白垩纪的原始力量。 史前荒野 陆地', '奔跑的奶龙 一只Q版的黄色小恐龙，有着大大的绿色眼睛和友善的微笑。是一部动画中的角色，非常可爱。 动画世界 未知', '驯龙高手：无牙仔 在电影《驯龙高手》中，主角小嗝嗝骑着他的龙伙伴无牙仔在高空飞翔。他们飞向灿烂的太阳，下方是岛屿和海洋，画面充满了冒险与友谊。 博克岛上空 天空', '中华金龙 一条金色的中华龙在祥云间盘旋，它身形矫健，龙须飘逸，展现了东方神话中龙的威严与神圣。 祥云之上 天空', '金丝猴 一只珍稀的金丝猴坐靠在树干上，悠闲地吃着食物。它拥有金色的长毛和蓝色的脸庞，是中国的特有物种。 森林 陆地']
--> 正在分批插入数据...
已经存在的数据数量: 6


`filter(function, iterable)` 的工作方式是：

- 对 `iterable`（这里是 `parts`）中的每个元素，调用 `function`。
- 如果 `function(element)` 返回 **真值（truthy）**，就保留该元素；否则丢弃。
- 当 `function` 是 `None` 时，`filter` 会自动使用“真假判断”：只保留 **真值（truthy）** 的元素。

在 Python 中，以下值被认为是 **假值（falsy）**：

- 空字符串 `''`
- `None`
- `0`
- 空列表 `[]`
- 空字典 `{}`
- 等等

In [ ]:
a = [ 1, 2, None, 4]
b = filter(None, a)
for s in b:
    print(s)

1
2
4


## 开始搜索

In [ ]:
search_query = "悬崖上的巨龙"

query_embeddings = ef([search_query])
print(query_embeddings)
print(len(query_embeddings['dense']))
print(ef._encode_config)

dense_vec = query_embeddings['dense'][0]
sparse_vec = query_embeddings['sparse']._getrow(0)


{'dense': [array([-0.00353044,  0.02043402, -0.04192595, ..., -0.02402728,
        0.00843867,  0.04209514], shape=(1024,), dtype=float32)], 'sparse': <Compressed Sparse Row sparse array of dtype 'float64'
	with 6 stored elements and shape (1, 250002)>}
1
{'batch_size': 16, 'return_dense': True, 'return_sparse': True, 'return_colbert_vecs': False}


- `sparse_vec` 是一个 **SciPy 稀疏向量**，通常是 `scipy.sparse.csr_matrix`、`csc_matrix` 或 `coo_matrix` 的一维形式（形状如 `(1, n)` 或 `(n, 1)`），或者是某些库（如 PyTorch Sparse、TensorFlow Sparse）中的稀疏表示。

- 在 SciPy 中，稀疏矩阵/向量常用 

  压缩格式

   存储，其中：

  - `.data`：存储所有**非零值**的数组。
  - `.indices`：存储对应非零值的**列索引**（对于 CSR/CSC 格式）。
  - `.nnz`：**non-zero count**，即非零元素的个数。

In [13]:
# 打印向量信息
print("\n=== 向量信息 ===")
print(f"密集向量维度: {len(dense_vec)}")
print(f"密集向量前5个元素: {dense_vec[:5]}")
print(f"密集向量范数: {np.linalg.norm(dense_vec):.4f}")

print(f"\n稀疏向量维度: {sparse_vec.shape[1]}")
print(f"稀疏向量非零元素数量: {sparse_vec.nnz}")
print("稀疏向量前5个非零元素:")
for i in range(min(5, sparse_vec.nnz)):
    print(f"  - 索引: {sparse_vec.indices[i]}, 值: {sparse_vec.data[i]:.4f}")
density = (sparse_vec.nnz / sparse_vec.shape[1] * 100)
print(f"\n稀疏向量密度: {density:.8f}%")


=== 向量信息 ===
密集向量维度: 1024
密集向量前5个元素: [-0.00353044  0.02043402 -0.04192595 -0.03036708 -0.02098165]
密集向量范数: 1.0000

稀疏向量维度: 250002
稀疏向量非零元素数量: 6
稀疏向量前5个非零元素:
  - 索引: 6, 值: 0.0659
  - 索引: 7977, 值: 0.1459
  - 索引: 14732, 值: 0.2959
  - 索引: 31433, 值: 0.1463
  - 索引: 141121, 值: 0.1587

稀疏向量密度: 0.00239998%


In [15]:
# 定义搜索参数
search_filter = 'category in ["western_dragon", "chinese_dragon", "movie_character"]'
top_k = 5
search_params = {"metric_type": "IP", "params": {}}
output_fields = ["title", "path", "description", "category", "location", "environment"]

## 单独-密集向量搜索

该字典用于配置向量相似度搜索的核心参数，主要包含两个关键字段：

1. `metric_type`（相似度度量类型）
   - `"IP"` 表示使用 **内积（Inner Product）** 作为相似度计算方式。
   - 内积适用于已归一化的向量（如通过 L2 归一化），此时内积等价于余弦相似度。
   - 其他常见取值：
     - `"L2"`：欧氏距离（值越小越相似）
     - `"COSINE"`：余弦相似度（需 Milvus 2.3+）
   - ⚠️ 注意：`metric_type` 必须与建索引时指定的度量类型一致，否则搜索结果无效。
2. `params`（索引特定搜索参数）
   - 用于传递底层索引算法所需的超参数，例如：
     - IVF 系列索引：`{"nprobe": 10}`（控制查询时扫描的聚类中心数）
     - HNSW 索引：`{"ef": 100}`（控制搜索时的候选节点数）
     - ANNOY 索引：`{"search_k": -1}`

报错：

1、

```
2026-02-04 08:07:25,355 [ERROR][handler]: RPC error: [search], <MilvusException: (code=101, message=failed to search: collection not loaded[collection=464044301868077488])>, <Time:{'RPC start': '2026-02-04 08:07:25.352769', 'RPC error': '2026-02-04 08:07:25.355006'}> (decorators.py:140)
```

根本原因：在 Milvus 中，集合（Collection）创建后必须显式加载到内存才能执行搜索操作。您尝试搜索时集合尚未加载，因此操作被拒绝。
 - 忘记调用 load


In [23]:

dense_results = collection.search([dense_vec], anns_field="dense_vector",
                                    limit=top_k, expr=search_filter,
                                    param=search_params, output_fields=output_fields)
dense_results = dense_results[0]

for i, hit in enumerate(dense_results):
    print(f"第 {i+1} 个结果. 标题={hit.entity.get('title')}. Score: {hit.distance}")
    print(f"    路径：{hit.entity.get('path')}")
    print(f"    描述：{hit.entity.get('description')}")

第 1 个结果. 标题=悬崖上的白龙. Score: 0.7213834524154663
    路径：../../data/C3/dragon/dragon02.png
    描述：一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。
第 2 个结果. 标题=中华金龙. Score: 0.5353410243988037
    路径：../../data/C3/dragon/dragon06.png
    描述：一条金色的中华龙在祥云间盘旋，它身形矫健，龙须飘逸，展现了东方神话中龙的威严与神圣。
第 3 个结果. 标题=驯龙高手：无牙仔. Score: 0.5230634212493896
    路径：../../data/C3/dragon/dragon05.png
    描述：在电影《驯龙高手》中，主角小嗝嗝骑着他的龙伙伴无牙仔在高空飞翔。他们飞向灿烂的太阳，下方是岛屿和海洋，画面充满了冒险与友谊。


## 单独-稀疏向量

In [26]:

sparse_results = collection.search([sparse_vec], anns_field="sparse_vector",
                                    limit=top_k, expr=search_filter,
                                    param=search_params, output_fields=output_fields)
sparse_results = sparse_results[0]

for i, hit in enumerate(sparse_results):
    print(f"第 {i+1} 个结果. 标题={hit.entity.get('title')}. Score: {hit.distance}")
    print(f"    路径：{hit.entity.get('path')}")
    print(f"    描述：{hit.entity.get('description')}")
    print(f"    位置：{hit.entity.get('location')}")
    print(f"    环境：{hit.entity.get('environment')}")

第 1 个结果. 标题=悬崖上的白龙. Score: 0.22544419765472412
    路径：../../data/C3/dragon/dragon02.png
    描述：一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。
    位置：悬崖海岸
    环境：天空
第 2 个结果. 标题=中华金龙. Score: 0.08569926023483276
    路径：../../data/C3/dragon/dragon06.png
    描述：一条金色的中华龙在祥云间盘旋，它身形矫健，龙须飘逸，展现了东方神话中龙的威严与神圣。
    位置：祥云之上
    环境：天空
第 3 个结果. 标题=驯龙高手：无牙仔. Score: 0.063875213265419
    路径：../../data/C3/dragon/dragon05.png
    描述：在电影《驯龙高手》中，主角小嗝嗝骑着他的龙伙伴无牙仔在高空飞翔。他们飞向灿烂的太阳，下方是岛屿和海洋，画面充满了冒险与友谊。
    位置：博克岛上空
    环境：天空


## 混合检索

In [27]:
rrf_rank = RRFRanker(k=60)

dense_req = AnnSearchRequest([dense_vec], "dense_vector", param=search_params, limit=top_k)
sparse_req =AnnSearchRequest([sparse_vec], "sparse_vector", param=search_params, limit=top_k)

results = collection.hybrid_search([sparse_req, dense_req], rerank=rrf_rank, 
                                    limit=top_k, output_fields=output_fields)

results = results[0]

for i, hit in enumerate(results):
    print(f"第 {i+1} 个结果. 标题={hit.entity.get('title')}. Score: {hit.distance}")
    print(f"    路径：{hit.entity.get('path')}")
    print(f"    描述：{hit.entity.get('description')}")
    print(f"    位置：{hit.entity.get('location')}")
    print(f"    环境：{hit.entity.get('environment')}")

第 1 个结果. 标题=悬崖上的白龙. Score: 0.032786883413791656
    路径：../../data/C3/dragon/dragon02.png
    描述：一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。
    位置：悬崖海岸
    环境：天空
第 2 个结果. 标题=中华金龙. Score: 0.0320020467042923
    路径：../../data/C3/dragon/dragon06.png
    描述：一条金色的中华龙在祥云间盘旋，它身形矫健，龙须飘逸，展现了东方神话中龙的威严与神圣。
    位置：祥云之上
    环境：天空
第 3 个结果. 标题=奔跑的奶龙. Score: 0.03151364624500275
    路径：../../data/C3/dragon/dragon04.png
    描述：一只Q版的黄色小恐龙，有着大大的绿色眼睛和友善的微笑。是一部动画中的角色，非常可爱。
    位置：动画世界
    环境：未知
第 4 个结果. 标题=驯龙高手：无牙仔. Score: 0.03125763311982155
    路径：../../data/C3/dragon/dragon05.png
    描述：在电影《驯龙高手》中，主角小嗝嗝骑着他的龙伙伴无牙仔在高空飞翔。他们飞向灿烂的太阳，下方是岛屿和海洋，画面充满了冒险与友谊。
    位置：博克岛上空
    环境：天空
第 5 个结果. 标题=霸王龙的怒吼. Score: 0.03125
    路径：../../data/C3/dragon/dragon03.png
    描述：史前时代的霸王龙张开血盆大口，发出震天的怒吼。在它身后，几只翼龙在阴沉的天空中盘旋，展现了白垩纪的原始力量。
    位置：史前荒野
    环境：陆地


## 清理资源

In [ ]:
milvus_client.release_collection(collection_name=COLLECTION_NAME)
print(f"已从内存中释放 Collection: '{COLLECTION_NAME}'")
milvus_client.drop_collection(COLLECTION_NAME)
print(f"已删除 Collection: '{COLLECTION_NAME}'")

## 练习

- 分析代码为什么在密集向量检索和稀疏向量检索中，排名第三的驯龙高手在混合检索中反而排在了第五？
  - 》： 因为缺少结果过滤，前2个都有过滤表达式。